In [12]:
import pandas as pd
import yake
from rake_nltk import Rake
import nltk
import spacy
import pytextrank

In [2]:
#!python -m spacy download en_core_web_sm
# nltk.download('stopwords')

In [5]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("textrank")
rake = Rake()

def extract_keywords_rake(text):
    rake.extract_keywords_from_text(text)
    return rake.get_ranked_phrases_with_scores()


def extract_keywords_textrank(text):
    doc = nlp(text)
    return [phrase.text for phrase in doc._.phrases[:10]]


In [6]:
df = pd.read_parquet('../data/merged_data.parquet')
hpb_df = df[df['pr_name'].fillna('').str.contains('Health Promotion Board')]
hpb_df = hpb_df[(hpb_df['to_remove'] == False) & (hpb_df['content_category'] == 'live-healthy-articles')]
hpb_df.head(3)
print(hpb_df.columns)
hpb_df.shape

Index(['id', 'content_name', 'title', 'article_category_names',
       'cover_image_url', 'full_url', 'full_url2', 'friendly_url',
       'category_description', 'content_body', 'keywords', 'feature_title',
       'pr_name', 'alternate_image_text', 'date_modified', 'number_of_views',
       'last_month_view_count', 'last_two_months_view', 'page_views',
       'engagement_rate', 'bounce_rate', 'exit_rate', 'scroll_percentage',
       'percentage_total_views', 'cumulative_percentage_total_views',
       'content_category', 'to_remove', 'has_table', 'has_image',
       'related_sections', 'extracted_links', 'extracted_headers',
       'extracted_content_body'],
      dtype='object')


(623, 33)

In [7]:
temp = hpb_df[['title','extracted_content_body']].copy()
yake_kw_extractor =  yake.KeywordExtractor(n=2, dedupLim=0.9, dedupFunc='seqm', windowsSize=1, top=10)

temp['yake_Title_KWs'] = temp['title'].apply(lambda x: yake_kw_extractor.extract_keywords(x))
temp['yake_Title_KWs_only'] = temp['yake_Title_KWs'].apply(lambda x: list(set([e[0].lower() for e in x])))

temp['yake_Content_KWs'] = temp['extracted_content_body'].apply(lambda x: yake_kw_extractor.extract_keywords(x))
temp['yake_Content_KWs_only'] = temp['yake_Content_KWs'].apply(lambda x: list(set([e[0].lower() for e in x])))

temp['rake_Content_KWs'] = temp['extracted_content_body'].apply(lambda x: extract_keywords_rake(x))
temp['rake_Content_KWs_only'] = temp['rake_Content_KWs'].apply(lambda x: list(set([e[1].lower() for e in x])))


In [14]:
temp[['title','extracted_content_body','yake_Title_KWs_only','yake_Content_KWs_only','rake_Content_KWs_only']]

,title,extracted_content_body,yake_Title_KWs_only,yake_Content_KWs_only,rake_Content_KWs_only
367,"Weight, BMI and Health Problems",What's a Healthy Body Mass Index?\nWe have all...,"[problems, bmi, health, weight, health problems]","[bmi, body mass, mass, healthy weight, index, ...","[get back, weight management, realistic bingin..."
368,7-month-baby Diet: An Authoritative Guide by O...,By Health Promotion Board in collaboration wit...,"[diet, authoritative, guide, authoritative gui...","[prof mary, children hospital, clinical servic...","[pumpkin, formula mid, babys meals may typical..."
369,Older Adults Need More Protein,"As you age, your body requires roughly 50% mor...","[adults, older adults, older, protein]","[muscle mass, protein, older adults, diet, mas...","[mix things, produced, whether youre eating, l..."
371,How Much Calories Do I Need A Day?,Figuring out Your Daily Calorie Intake Isnt Ro...,"[calories, day]","[calories, daily calorie, intake, healthy, cal...","[daily target, large, overeating, look, range,..."
374,All You Need to Know About Childhood Immunisat...,Every child in Singapore is vaccinated accordi...,"[childhood immunisations, childhood, immunisat...","[child, childhood immunisation, childhood, imm...","[applicable, altered, side effects, answers, s..."
...,...,...,...,...,...
1516,Getting the Fats Right!,Fat is not all bad...\nFat plays an important ...,[fats],"[trans fat, oil, products, fat, saturated fat,...","[products containing vegetable, foods, fortifi..."
1517,Kashmiri Pulao,A fragrant brown rice dish with raisins\n\n\nI...,"[kashmiri pulao, kashmiri, pulao]","[tsp, cups water, cups long, salt method, grai...","[heat oil, sauteed spices, cover pot tightly, ..."
1519,Healthy Food for Kids and Teens,Meeting nutritional needs of kids and teenager...,"[teens, healthy food, healthy, kids, food]","[child, healthier oils, child diet, vegetables...","[excessive weight gain, nutritional value, who..."
1520,How to Study Difficult Subjects,So you're pretty happy with the way you've bee...,"[study difficult, difficult, difficult subject...","[taking regular, easier, make, thing, regular ...","[many different fields, another thing, schedul..."
